# Modelo genérico



## Índices / Conjuntos

$I \colon \{1,\ldots, m\}$

$J \colon \{1,\ldots, n\}$

## Parâmetros

$d_{ij} \colon  \text{distância do ponto } i \in I \text{ até o ponto } j \in J$

$p = \text{número de medianas}$


## Variáveis de decisão

$
    y_{j}=
    \begin{cases}
      1, & \text{se o ponto } j \in J \text{ é escolhido como mediana} \\
      0, & \text{caso contrário}
    \end{cases}
$

$
    x_{ij}=
    \begin{cases}
      1, & \text{se o ponto } i \in I \text{ é alocado à mediana } j \in J \\
      0, & \text{caso contrário}
    \end{cases}
$

## Formulação matemática

$\text{min }z = \sum\limits_{i \in I} \sum\limits_{j \in J} d_{ij} x_{ij}$

sujeito a

$\sum\limits_{j \in J} y_{j} = p$

$\sum\limits_{j \in J} x_{ij} = 1, \;\; \forall i \in I$

$x_{ij} \leq y_j, \;\; \forall i \in I,j \in J$

$y_j \in \{0,1\}, \;\; \forall \; j \in J$

$x_{ij} \in \{0,1\}, \;\; \forall i \in I, j \in J$

In [ ]:
# Carrega as bibliotecas necessárias:

import pyomo.environ as pyo
import pandas as pd

In [ ]:
# Declara o modelo computacional:

modelo = pyo.AbstractModel()

# Parâmetros auxiliares:
modelo.m = pyo.Param()
modelo.n = pyo.Param()
modelo.distancias = pyo.Param(within=pyo.Any)

# Índices:
modelo.I = pyo.RangeSet(modelo.m)
modelo.J = pyo.RangeSet(modelo.n)

# Parâmetros:
modelo.d = pyo.Param(modelo.I, modelo.J, initialize=lambda modelo, i, j: modelo.distancias()[i-1][j-1])
modelo.p = pyo.Param()

# Variáveis de decisão:
modelo.y = pyo.Var(modelo.J, within=pyo.Binary)
modelo.x = pyo.Var(modelo.I, modelo.J, within=pyo.Binary)

# Função objetivo:
def f_obj(modelo):
    return sum(modelo.x[i,j] * modelo.d[i,j] for i in modelo.I for j in modelo.J)
modelo.z = pyo.Objective(rule=f_obj, sense=pyo.minimize)

# Restrições:

def f_restr1(modelo, j):
    return sum(modelo.y[j] for j in modelo.J) == modelo.p()
modelo.restr_1 = pyo.Constraint(modelo.J, rule=f_restr1)

def f_restr2(modelo, i):
    return sum(modelo.x[i,j] for j in modelo.J) == 1
modelo.restr_2 = pyo.Constraint(modelo.I, rule=f_restr2)

def f_restr3(modelo, i, j):
    return modelo.x[i,j] <= modelo.y[j]
modelo.restr_3 = pyo.Constraint(modelo.I, modelo.J, rule=f_restr3)

In [ ]:
# Lê os dados de entrada:

dados_entrada = pyo.DataPortal()
dados_entrada.load(filename='dados_entrada.json')

In [ ]:
# Cria a instância:

instancia = modelo.create_instance(dados_entrada)

In [ ]:
instancia.pprint()

In [ ]:
# Resolve a instância:

resultado = pyo.SolverFactory('glpk').solve(instancia)
print(resultado)

In [ ]:
instancia.y.pprint()

In [ ]:
instancia.x.pprint()